# Tracer Advection Stencil

#### 1. Start a parallel cluster

In [1]:
from functions import show_clusters

import ipyparallel as ipp
import time

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()


Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

In [3]:
time.sleep(15)
show_clusters()
%autopx

cluster_id        state    cluster_file
1657815598-o3vt  running   /home/ajdas/.ipython/profile_default/security/cluster-1657815598-o3vt.json
%autopx enabled


#### 2. Configure the domain

In [243]:
importlib.reload(func)
#%%capture
from mpi4py import MPI
import functions as func
import importlib
importlib.reload(func)


mpi_comm = MPI.COMM_WORLD
mpi_size = mpi_comm.Get_size()
mpi_rank = mpi_comm.Get_rank()

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 100, 100, 79, 3
dimensions = {'nx': nx, 'ny': ny, 'nx1': nx+1, 'ny1': ny+1, 'nz': nz, 'nhalo': nhalo, 'tile': mpi_size, 'nxhalo': nx+2*nhalo, 'nyhalo': ny+2*nhalo}
units = {'dist': 'm', 'coord': 'degrees', 'qvapor': 'kg/kg', 'psi': 'kg/m/s', 'wind': 'm/s', 'courant': '', 'areaflux': 'm2', 'qflux': 'kg/m2'}
origins = {'halo': (0, 0), 'compute_2d': (dimensions['nhalo'], dimensions['nhalo']), 'compute_3d': (dimensions['nhalo'], dimensions['nhalo'], 0)}


configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)

[stderr:2] /home/ajdas/pace/pace-util/pace/util/grid/gnomonic.py:681: RuntimeWarning: invalid value encountered in true_divide
  np.sum(p * q, axis=-1)


[stderr:5] /home/ajdas/pace/pace-util/pace/util/grid/gnomonic.py:681: RuntimeWarning: invalid value encountered in true_divide
  np.sum(p * q, axis=-1)


#### 2a. Gather domain coordinates

In [244]:
from pace.util import Quantity
import numpy as np

dx_halo =  Quantity(configuration['grid_data'].dx.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dy_halo =  Quantity(configuration['grid_data'].dy.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dxa_halo =  Quantity(configuration['grid_data'].dxa.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dya_halo =  Quantity(configuration['grid_data'].dya.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dxc_halo =  Quantity(configuration['grid_data'].dxc.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dyc_halo =  Quantity(configuration['grid_data'].dyc.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dx_global = configuration['communicator'].gather(dx_halo)
dy_global = configuration['communicator'].gather(dy_halo)
dxa_global = configuration['communicator'].gather(dxa_halo)
dya_global = configuration['communicator'].gather(dya_halo)
dxc_global = configuration['communicator'].gather(dxc_halo)
dyc_global = configuration['communicator'].gather(dyc_halo)

lona_halo =  Quantity(configuration['grid_data'].lon_agrid.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lata_halo =  Quantity(configuration['grid_data'].lat_agrid.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lona_global = configuration['communicator'].gather(lona_halo)
lata_global = configuration['communicator'].gather(lata_halo)

#### 2b. Save coordinates to netCDF file

Convert below markdown cell to code.

```
import functions as func


lon_halo =  Quantity(configuration['grid_data'].lon.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lat_halo =  Quantity(configuration['grid_data'].lat.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)


lon_global = configuration['communicator'].gather(lon_halo)
lat_global = configuration['communicator'].gather(lat_halo)

fOut = 'coordinates.nc'
variables = {'dx': dx_global, 'dy': dy_global, 
             'dxa': dxa_global, 'dya': dya_global, 
             'dxc': dxc_global, 'dyc': dyc_global, 
             'lon': lon_global, 'lat': lat_global, 
             'lona': lona_global, 'lata': lata_global}

if mpi_rank == 0:
    func.store_coordinates(fOut, dimensions, variables)
```

#### 3. Create initial conditions on a plane

#### 3a. Create water vapor initial condition

In [261]:
if mpi_rank == 0:
    gaussian_multiplier = func.create_gaussianMultiplier(np.deg2rad(lona_global.data), np.deg2rad(lata_global.data), dimensions, center_tile=0)
else:
    gaussian_multiplier = np.zeros((dimensions['tile'], dimensions['nxhalo'], dimensions['nyhalo']))

mpi_comm.Bcast(gaussian_multiplier, root=0)
qvapor = Quantity(gaussian_multiplier[mpi_rank], ('x', 'y'), units['qvapor'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
qvapor_global = configuration['communicator'].gather(qvapor)

[stdout:0] Centering gaussian on lon=351.01, lat=1.01


#### 3b. Run coordinates through streamfunction generation

In [262]:
psi, psi_staggered = func.calculate_streamfunction_testCase1(np.deg2rad(lona_halo.data), np.deg2rad(lata_halo.data), dimensions)

psi_halo = Quantity(psi, ('x_halo', 'y_halo'), units['psi'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
psi = Quantity(psi, ('x', 'y'), units['psi'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
psi_staggered_halo = Quantity(psi_staggered, ('x_halo', 'y_halo'), units['psi'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
psi_staggered  = Quantity(psi_staggered, ('x_interface', 'y_interface'), units['psi'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)

psi_global = configuration['communicator'].gather(psi)
psi_staggered_global = configuration['communicator'].gather(psi_staggered)

#### 3c. Calculate winds from streamfunction

`ua`, `va` - winds on cell centers
`uc`, `vc` - covariant winds on C-grid

In [263]:
ua, va = func.calculate_windsFromStreamfunction_grid(psi_halo, dxa_halo, dya_halo, dimensions, grid='A')

ua = Quantity(ua, ('x', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
va = Quantity(va, ('x', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)

ua_global = configuration['communicator'].gather(ua)
va_global = configuration['communicator'].gather(va)

In [264]:
uc, vc = func.calculate_windsFromStreamfunction_grid(psi_staggered_halo, dx_halo, dy_halo, dimensions, grid='C')

uc = Quantity(uc, ('x', 'y_interface'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny1']), backend)
vc = Quantity(vc, ('x_interface', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny']), backend)

uc_global = configuration['communicator'].gather(uc)
vc_global = configuration['communicator'].gather(vc)

In [265]:
ud, vd = func.calculate_windsFromStreamfunction_grid(psi_halo, dx_halo, dy_halo, dimensions, grid='D')

ud = Quantity(ud, ('x_interface', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny']), backend)
vd = Quantity(vd, ('x', 'y_interface'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny1']), backend)

ud_global = configuration['communicator'].gather(ud)
vd_global = configuration['communicator'].gather(vd)

#### 3d. Find pressure distribution consistent with velocity field ?????

$$
\nabla ^2 p = 2 \rho \left[ \frac{\partial u}{\partial x} \frac{\partial v}{\partial y} - \frac{\partial v}{\partial x} \frac{\partial u}{\partial y} \right]
$$

#### 3d. Plot initial conditions and save data to file
To show plotted images, uncomment the second cell below

In [266]:
# %%capture capt --no-stderr --no-stdout
# fOut = 'initialize_winds.nc'

# lona =  Quantity(configuration['grid_data'].lon_agrid.data * 180 / np.pi, ('x_interface', 'y_interface'), units['coord'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)
# lata =  Quantity(configuration['grid_data'].lat_agrid.data * 180 / np.pi, ('x_interface', 'y_interface'), units['coord'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)
# lon_global = configuration['communicator'].gather(lona)
# lat_global = configuration['communicator'].gather(lata)

# if mpi_rank == 0:
#     variables = {'lon': lon_global.data, 'lat': lat_global.data, 'ua': ua_global.data, 'va': va_global.data, 'psi': psi_global.data, 'qvapor': qvapor_global.data}

#     func.plot_projection_field(variables['lon'], variables['lat'], variables['qvapor'], vmin=-0, vmax=1, cmap='viridis', title='q_vapor init', units=units['qvapor'])
#     func.plot_projection_field(variables['lon'], variables['lat'], variables['ua'], vmin=-40, vmax=40, cmap='bwr', title='ua init', units=units['wind'])
#     func.plot_projection_field(variables['lon'], variables['lat'], variables['va'], vmin=-40, vmax=40, cmap='bwr', title='va init', units=units['wind'])
#     func.plot_projection_field(variables['lon'], variables['lat'], variables['psi'], vmin=-3e8, vmax=3e8, cmap='bwr', title='psi init', units=units['psi'])

#     func.write_initialCondition_toFile(fOut, variables, dimensions, units)

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

In [267]:
# capt()

#### 4. Prepare flux stencil

#### 4a. Extend initial conditions into the vertical

In [268]:
dimensions['nz'] = 1

empty = np.zeros((dimensions['nxhalo']+1, dimensions['nyhalo']+1, dimensions['nz']+1))

qvapor_3d = np.copy(empty)
uc_3d = np.copy(empty)
vc_3d = np.copy(empty)
qvapor_3d[:-1, :-1, 0] = qvapor
uc_3d[:-1, :-1, 0] = uc
vc_3d[:-1, :-1, 0] = vc

qvapor = Quantity(qvapor_3d, ('x', 'y', 'z'), units['qvapor'], origins['compute_3d'], (dimensions['nx'], dimensions['ny'], dimensions['nz']), backend)
uc = Quantity(uc_3d, ('x', 'y_interface', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)
vc = Quantity(vc_3d, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)

#### 4b. Prepare data for flux calculation

- use the function `FiniteVolumeFluxPrep`
    - requires `uc` and `vc` from (3c). 
    - requires some empty arrays to be filled in:
        - `crx` and `cry` are the dimensionless Courant numbers on C-grid points.
        - `xaf` and `yaf` are are fluxes of area in m2.
        - `ucc` and `vcc` are the contravariant velocities on C-grid points
        - `dt` is the acoustic time step.

In [207]:
%%capture capt
from fv3core.stencils.fxadv import FiniteVolumeFluxPrep

fvf_prep = FiniteVolumeFluxPrep(configuration['stencil_factory'], configuration['grid_data'])

crx = Quantity(empty, ('x_interface', 'y', 'z'), units['courant'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
cry = Quantity(empty, ('x', 'y_interface', 'z'), units['courant'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)

xaf = Quantity(empty, ('x_interface', 'y', 'z'), units['areaflux'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
yaf = Quantity(empty, ('x', 'y_interface', 'z'), units['areaflux'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)

ucv = Quantity(empty, ('x_interface', 'y', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)
vcv = Quantity(empty, ('x', 'y_interface', 'z'), units['wind'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)


dt_acoustic = 300.

fvfp = fvf_prep(uc, vc, crx, cry, xaf, yaf, ucv, vcv, dt_acoustic)

print()
print('UC min, mean, max: %.2f %.2f %.2f' % (np.nanmin(uc), np.nanmean(uc), np.nanmax(uc)))
print('UCV min, mean, max: %.2f %.2f %.2f' % (np.nanmin(ucv), np.nanmean(ucv), np.nanmax(ucv)))
print()
print('VC min, mean, max: %.2f %.2f %.2f' % (np.nanmin(vc), np.nanmean(vc), np.nanmax(vc)))
print('VCV min, mean, max: %.2f %.2f %.2f' % (np.nanmin(vcv), np.nanmean(vcv), np.nanmax(vcv)))
print()
print('X-Courant min, mean, max: %.2f %.2f %.2f' % (np.nanmin(crx), np.nanmean(crx), np.nanmax(crx)))
print('Y-Courant min, mean, max: %.2f %.2f %.2f' % (np.nanmin(cry), np.nanmean(cry), np.nanmax(cry)))
print()
print('X-area flux min, mean, max: %.2f %.2f %.2f' % (np.nanmin(xaf), np.nanmean(xaf), np.nanmax(xaf)))
print('X-area flux min, mean, max: %.2f %.2f %.2f' % (np.nanmin(yaf), np.nanmean(yaf), np.nanmax(yaf)))

#### (4c) Hack mass flux?

xaf is the flux of area in x direction (dx*dy)
yaf is the flux of area in y direction (dx*dy)

to get mass flux -- 
for now -- assume mass flux is area flux.

#### (4d) Create finite volume transport stencil

- use the function `FiniteVolumeTransport` 
    - takes in the tracer `qvap` to be transported.
    - takes in the `crx` and `cry` Courant numbers, `xaf` and `yaf` area fluxes from previous step.
    - requires some empty arrays to be filled in:
        - `qfx` and `qfy` are the output fluxes of the tracer to be transported -- i think kg/m2?
        
- also takes in mass fluxes (optional) -- where would I find those?

- do I actually need the `qfx`, `qfy` that are output? I don't think so ...

In [208]:
#%%capture capt
from fv3core.stencils.fvtp2d import FiniteVolumeTransport

grid_type = 0
hord = 6

qfx = Quantity(empty, ('x', 'y_interface', 'z'), units['qflux'], origins['compute_3d'], (dimensions['nx'], dimensions['ny1'], dimensions['nz']), backend)
qfy = Quantity(empty, ('x_interface', 'y', 'z'), units['qflux'], origins['compute_3d'], (dimensions['nx1'], dimensions['ny'], dimensions['nz']), backend)

tracers = {'qvapor': qvapor}

fvtp2d = FiniteVolumeTransport(configuration['stencil_factory'], configuration['grid_data'], configuration['damping_coefficients'], grid_type, hord)
fvTransport = fvtp2d(tracers['qvapor'], crx, cry, xaf, yaf, qfx, qfy)

[5:execute]
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [202], in <cell line: 13>()
     10 tracers = {'qvapor': qvapor}
     12 fvtp2d = FiniteVolumeTransport(configuration['stencil_factory'], configuration['grid_data'], configuration['damping_coefficients'], grid_type, hord)
---> 13 fvTransport = fvtp2d(tracers['qvapor'], crx, cry, xaf, yaf, qfx, qfy)

File ~/pace/fv3core/fv3core/stencils/fvtp2d.py:300, in FiniteVolumeTransport.__call__(self, q, crx, cry, x_area_flux, y_area_flux, q_x_flux, q_y_flux, x_mass_flux, y_mass_flux, mass)
    294     y_unit_flux = y_mass_flux
    296 # TODO: consider whether to refactor xppm/yppm to output fluxes by also taking
    297 # y_area_flux as an input (flux = area_flux * advected_mean), since a flux is
    298 # easier to understand than the current output. This would be like merging
    299 # yppm with q_i_stencil and xppm with q_j

AlreadyDisplayedError: 6 errors

In [223]:
tracers['qvapor'].metadata

Out[0:216]: QuantityMetadata(origin=(3, 3, 0), extent=(100, 100, 1), dims=('x', 'y', 'z'), units='kg/kg', data_type=<class 'numpy.ndarray'>, dtype=dtype('float64'), gt4py_backend='numpy')

Out[3:216]: QuantityMetadata(origin=(3, 3, 0), extent=(100, 100, 1), dims=('x', 'y', 'z'), units='kg/kg', data_type=<class 'numpy.ndarray'>, dtype=dtype('float64'), gt4py_backend='numpy')

Out[5:216]: QuantityMetadata(origin=(3, 3, 0), extent=(100, 100, 1), dims=('x', 'y', 'z'), units='kg/kg', data_type=<class 'numpy.ndarray'>, dtype=dtype('float64'), gt4py_backend='numpy')

Out[1:216]: QuantityMetadata(origin=(3, 3, 0), extent=(100, 100, 1), dims=('x', 'y', 'z'), units='kg/kg', data_type=<class 'numpy.ndarray'>, dtype=dtype('float64'), gt4py_backend='numpy')

Out[4:216]: QuantityMetadata(origin=(3, 3, 0), extent=(100, 100, 1), dims=('x', 'y', 'z'), units='kg/kg', data_type=<class 'numpy.ndarray'>, dtype=dtype('float64'), gt4py_backend='numpy')

Out[2:216]: QuantityMetadata(origin=(3, 3, 0), extent=(100, 100, 1), dims=('x', 'y', 'z'), units='kg/kg', data_type=<class 'numpy.ndarray'>, dtype=dtype('float64'), gt4py_backend='numpy')

In [ ]:
import importlib
importlib.reload(func)

In [ ]:
# fInit = 'initialize_advection.nc'

# if mpi_rank == 0:
#     %run -i initialize_streamfunction_advection.py $fOut $fInit